In [2]:
import mysql.connector
conn=mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="atm"
)
cursor=conn.cursor()

create_table= """
CREATE TABLE IF NOT EXISTS USER (
    ID INT ,
    NAME VARCHAR(30) NOT NULL,
    ACCOUNT_NO INT PRIMARY KEY,
    PASSWORD INT NOT NULL 
)"""
cursor.execute(create_table)

create_table1 = """
CREATE TABLE IF NOT EXISTS TRANSACTIONS (
    ACCOUNT_NO INT,
    FOREIGN KEY (ACCOUNT_NO) REFERENCES USER(ACCOUNT_NO),
    TRANSACTION_ID INT PRIMARY KEY AUTO_INCREMENT,
    DEPOSITS INT,
    WITHDRAWLS INT,
    CURRENT_BALANCE INT
)"""
cursor.execute(create_table1)


print("Welcome to HDFC  ATM")
print("Please Insert your Debit Card")
print("Plese Wait For Some Time While Checking Your Card Details")


def insert_query():
    id1=int(input("ENTER YOUR ID NO."))
    name1=input("ENTER YOUR NAME:")
    account_no1=int(input("ENTER YOUR 10 DIGIT ACCOUNT NO."))
    password1=int(input("ENTER YOUR 4 DIGIT PIN HERE:"))
    insert_data = """
    insert into USER (ID, NAME, ACCOUNT_NO, PASSWORD) VALUES (%s,%s,%s,%s)
    """
    cursor.execute(insert_data,(id1,name1,account_no1,password1))
    print("DATA INSERTED SUCCESFULLY")
    conn.commit()
    

def balance_check():
    select_query_balance="""SELECT CURRENT_BALANCE FROM TRANSACTIONS WHERE ACCOUNT_NO=%s """
    cursor.execute(select_query_balance,(user,))
    balance=cursor.fetchone()
    print(f"Updated Balance: {balance}")



def deposit():
    deposit = int(input("ENTER AMOUNT YOU WANT TO DEPOSIT:"))
    deposit_query = """
    UPDATE TRANSACTIONS 
    SET DEPOSITS = COALESCE(DEPOSITS, 0) + %s 
    WHERE ACCOUNT_NO = %s;
    """
    cursor.execute(deposit_query, (deposit,user))
    conn.commit()
    current_query = """
        SELECT CURRENT_BALANCE 
        FROM TRANSACTIONS 
        WHERE ACCOUNT_NO = %s;
    """
    cursor.execute(current_query,(user,))
    result = cursor.fetchone()
    if result:
        current_balance = result[0] + deposit
        update_balance_query = """
            UPDATE TRANSACTIONS 
            SET CURRENT_BALANCE = %s 
            WHERE ACCOUNT_NO = %s;
        """
        cursor.execute(update_balance_query, (current_balance,user))
        conn.commit()

        print(f"Updated Balance: {current_balance}")
    else:
        print("Account not found!")



def withdraw():
    withdraw = int(input("ENTER AMOUNT YOU WANT TO WITHDRAW:"))
    withdraw_query = """
    UPDATE TRANSACTIONS 
    SET WITHDRAWLS = COALESCE(WITHDRAWLS, 0) + %s 
    WHERE ACCOUNT_NO = %s;
    """
    cursor.execute(withdraw_query, (withdraw,user))
    conn.commit()
    current_query = """
        SELECT CURRENT_BALANCE 
        FROM TRANSACTIONS 
        WHERE ACCOUNT_NO = %s;
    """
    cursor.execute(current_query,(user,))
    result = cursor.fetchone()
    if result:
        current_balance = result[0] - withdraw
        update_balance_query = """
            UPDATE TRANSACTIONS 
            SET CURRENT_BALANCE = %s 
            WHERE ACCOUNT_NO = %s;
        """
        cursor.execute(update_balance_query, (current_balance,user))
        conn.commit()

        print(f"Updated Balance: {current_balance}")
    else:
        print("Account not found!")


user=int(input("ENTER YOUR 10 DIGIT ACCOUNT NO."))
pin=int(input("ENTER YOUR 4 DIGIT PIN HERE:"))
select_query="""SELECT * FROM USER WHERE ACCOUNT_NO=%s AND PASSWORD=%s """
cursor.execute(select_query,(user,pin))
l=cursor.fetchone()
if l:
    print("LOGIN SUCCESSFULLY")

    print("OPTIONS= WITHDRAW CASH, CHECK BALANCE, DEPOSIT CASH")
    a=int(input(" FOR WITHDRAW MONEY CHOOSE OPTION 1,\n FOR DEPOSIT MONEY CHOOSE OPTION 2 \n FOR BALANCE CHECK CHOOSE OPTION 3"))

    if a==1:
        withdraw()
    elif a==2:
        deposit()
    elif a==3:
        balance_check()
    else:
        print("INVALID INPUT\n PLESE TRY AGAIN!") 
else:
    print("INVALID DETAILS")

Welcome to HDFC  ATM
Please Insert your Debit Card
Plese Wait For Some Time While Checking Your Card Details


ENTER YOUR 10 DIGIT ACCOUNT NO. 2024111402
ENTER YOUR 4 DIGIT PIN HERE: 4321


LOGIN SUCCESSFULLY
OPTIONS= WITHDRAW CASH, CHECK BALANCE, DEPOSIT CASH


 FOR WITHDRAW MONEY CHOOSE OPTION 1,
 FOR DEPOSIT MONEY CHOOSE OPTION 2 
 FOR BALANCE CHECK CHOOSE OPTION 3 3


Updated Balance: (40000,)


In [ ]:
import mysql.connector
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="atm"
)
cursor = conn.cursor()

# Create USER table with AUTO_INCREMENT for ID
create_table = """
CREATE TABLE IF NOT EXISTS USER (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    NAME VARCHAR(30) NOT NULL,
    ACCOUNT_NO INT UNIQUE NOT NULL,
    PASSWORD INT NOT NULL
)"""
cursor.execute(create_table)

# Create TRANSACTIONS table with separate transaction records
create_table1 = """
CREATE TABLE IF NOT EXISTS TRANSACTIONS (
    TRANSACTION_ID INT PRIMARY KEY AUTO_INCREMENT,
    ACCOUNT_NO INT,
    TRANSACTION_TYPE VARCHAR(20),
    AMOUNT INT,
    CURRENT_BALANCE INT,
    FOREIGN KEY (ACCOUNT_NO) REFERENCES USER(ACCOUNT_NO)
)"""
cursor.execute(create_table1)

# Function to insert a new user
def insert_query():
    name1 = input("ENTER YOUR NAME: ")
    account_no1 = int(input("ENTER YOUR 10 DIGIT ACCOUNT NO.: "))
    password1 = int(input("ENTER YOUR 4 DIGIT PIN HERE: "))
    insert_data = """
    INSERT INTO USER (NAME, ACCOUNT_NO, PASSWORD) VALUES (%s, %s, %s)
    """
    cursor.execute(insert_data, (name1, account_no1, password1))
    conn.commit()
    print("DATA INSERTED SUCCESSFULLY")

# Function to check balance
def balance_check(user):
    select_query_balance = """SELECT CURRENT_BALANCE FROM TRANSACTIONS WHERE ACCOUNT_NO=%s ORDER BY TRANSACTION_ID DESC LIMIT 1"""
    cursor.execute(select_query_balance, (user,))
    balance = cursor.fetchone()
    if balance:
        print(f"Updated Balance: {balance[0]}")
    else:
        print("No transaction history found.")

# Function to deposit money
def deposit(user):
    deposit = int(input("ENTER AMOUNT YOU WANT TO DEPOSIT: "))
    current_query = """SELECT CURRENT_BALANCE FROM TRANSACTIONS WHERE ACCOUNT_NO=%s ORDER BY TRANSACTION_ID DESC LIMIT 1"""
    cursor.execute(current_query, (user,))
    result = cursor.fetchone()
    new_balance = result[0] + deposit if result else deposit  # If no previous transaction, set balance as deposit
    transaction_query = """
    INSERT INTO TRANSACTIONS (ACCOUNT_NO, TRANSACTION_TYPE, AMOUNT, CURRENT_BALANCE)
    VALUES (%s, 'DEPOSIT', %s, %s)
    """
    cursor.execute(transaction_query, (user, deposit, new_balance))
    conn.commit()
    print(f"Updated Balance: {new_balance}")

# Function to withdraw money
def withdraw(user):
    withdraw = int(input("ENTER AMOUNT YOU WANT TO WITHDRAW: "))
    current_query = """SELECT CURRENT_BALANCE FROM TRANSACTIONS WHERE ACCOUNT_NO=%s ORDER BY TRANSACTION_ID DESC LIMIT 1"""
    cursor.execute(current_query, (user,))
    result = cursor.fetchone()
    if result and result[0] >= withdraw:
        new_balance = result[0] - withdraw
        transaction_query = """
        INSERT INTO TRANSACTIONS (ACCOUNT_NO, TRANSACTION_TYPE, AMOUNT, CURRENT_BALANCE)
        VALUES (%s, 'WITHDRAWAL', %s, %s)
        """
        cursor.execute(transaction_query, (user, withdraw, new_balance))
        conn.commit()
        print(f"Updated Balance: {new_balance}")
    else:
        print("Insufficient balance or account not found.")

# Main program
user = int(input("ENTER YOUR 10 DIGIT ACCOUNT NO.: "))
pin = int(input("ENTER YOUR 4 DIGIT PIN HERE: "))
select_query = """SELECT * FROM USER WHERE ACCOUNT_NO=%s AND PASSWORD=%s"""
cursor.execute(select_query, (user, pin))
l = cursor.fetchone()

if l:
    print("LOGIN SUCCESSFUL")
    print("OPTIONS: WITHDRAW CASH, CHECK BALANCE, DEPOSIT CASH")
    a = int(input("FOR WITHDRAW MONEY CHOOSE OPTION 1,\nFOR DEPOSIT MONEY CHOOSE OPTION 2,\nFOR BALANCE CHECK CHOOSE OPTION 3: "))

    if a == 1:
        withdraw(user)
    elif a == 2:
        deposit(user)
    elif a == 3:
        balance_check(user)
    else:
        print("INVALID INPUT, PLEASE TRY AGAIN!")
else:
    print("INVALID DETAILS")
